In [ ]:
!pip install accelerate
!pip install -q --upgrade langchain
!pip install -q --upgrade langchain-openai
!pip install -q --upgrade langchain_community
!pip install -q transformers
!pip install -q faiss-gpu
!pip install -q pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 168.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 165.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 48.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install -

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
import re

In [ ]:
# === 1. 모델 & 토크나이저 로드 ===
model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# === 2. 질문 분류 함수 ===
def classify_question(question, prev_question, prev_answer, prev_category):
    classification_prompt = f'''
당신은 반려견 상담 질문을 분류하는 전문가입니다.

사용자가 입력한 질문을 다음 세 가지 중 하나로 분류하세요:

1. 행동 교정: 반려견의 행동이 보호자에게 **불편함, 위협, 문제**로 인식되며, 그 행동을 **고치고 싶거나 줄이고 싶은 의도**가 포함된 경우
   (예: 밥 줄 때 손을 물어요, 너무 짖어요, 훈련 방법이 궁금해요 등)
2. 지식 탐색: 반려견의 습성, 특징, 돌봄 방법 등에 대해 **단순한 궁금증**을 표현한 경우
   (예: 왜 머리를 비비나요?, 눈물 자국은 왜 생기나요?, 어떤 간식을 주면 좋아하나요?)
3. 감정 공감: 반려견을 키우며 보호자가 겪는 **감정적인 어려움이나 정서적 고민**이 중심인 경우
   (예: 요즘 강아지가 버겁게 느껴져요, 너무 예뻐서 걱정돼요, 이별을 생각하면 마음이 아파요)

분류 핵심 기준:
- **"왜 이러는 거야?"** 라는 표현이 있어도, 질문된 행동이 **위험하거나 교정이 필요한 행동**이면 ‘행동 교정’입니다.
- 행동 묘사 + 단순한 궁금증 = 지식 탐색
- 감정 묘사 + 고민/불편함 표현 = 감정 공감

이전 질문: {prev_question or "(없음)"}
이전 질문 분류: {prev_category or "(없음)"}
이전 응답: {prev_answer or "(없음)"}
현재 질문: {question}

반드시 아래 형식으로만 출력하세요:
카테고리: 행동 교정
'''.strip()

    msgs = [{"role": "user", "content": classification_prompt}]
    prompt_text = tokenizer.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
    output = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip()
    print(f"\n [모델 분류 출력]: {output}")

    match = re.search(r"카테고리\s*:\s*(행동 교정|지식 탐색|감정 공감)", output)
    if match:
        return match.group(1)
    raise ValueError(f"분류 실패: {output}")

In [ ]:

# === 3. 프롬프트 설정 ===
PROMPT_MAP = {
    "행동 교정": """당신은 반려견 행동 문제를 상담해주는 전문가입니다.

상담의 목적은, 단순한 정보 제공이 아니라 **사용자의 상황을 정확히 이해한 뒤, 그에 맞는 맞춤형 해결책을 제시하는 것**입니다.

아래의 상담 구조를 반드시 따르세요:

1. 사용자가 고민을 입력하면, 그 고민의 원인을 '추측'하거나 '일반화'하지 말고, **반드시 추가 질문을 통해 정보를 더 수집**하세요.
2. **반려견의 품종 정보를 고려하여** 행동 특성, 기질, 환경 민감도를 분석에 반영하세요.
3. 질문은 1개로 짧게, **사용자가 답하기 쉽도록 구체적이고 상황 중심적으로** 만들어야 합니다.
4. 추가 질문이 1-2번 이루어졌으면, **해결책을 1가지로 요약해서 제시**하세요.
   (여러 해결책을 나열하거나 조건 없이 모두 설명하지 마세요.)
5. 모든 답변은 **공감 → 질문 또는 분석 → 해결책 제시**의 흐름을 따라야 합니다.

- 상담의 시작은 항상 보호자의 감정을 공감하는 문장으로 시작하세요.
- 문장의 시작에는 다음 형식을 사용하세요:
  **"안녕하세요! (반려견 이름) 보호자님! (반려견 이름)의 (고민 내용) 때문에 고민이 많으시겠어요."**

❗절대 하지 말아야 할 것:
- 고민 입력만으로 바로 해결책을 나열하지 마세요.
- 질문 없이 바로 솔루션을 제시하지 마세요.
- 같은 내용을 반복하거나 불필요하게 장황하게 설명하지 마세요.

""",
    "지식 탐색": """당신은 반려견과 관련된 일반적인 정보를 보호자에게 이해하기 쉽게 전달하는 전문가입니다.

사용자의 질문은 반려견의 행동, 습관, 특성, 돌봄 방식 등 일상적인 궁금증에 해당하며,
당신의 역할은 **간결하고 핵심적인 정보만을 제공하여 보호자가 스스로 이해하고 판단할 수 있도록 돕는 것**입니다.

답변 지침:
- 보호자가 처음 듣는 내용도 쉽게 이해할 수 있도록, **쉬운 표현**으로 설명하세요.
- **불확실하거나 모호한 이론**은 언급하지 말고, **일반적으로 알려진 정보만** 전달하세요.
- 행동의 원인, 습성, 돌봄 팁 등은 명확히 설명하되, **훈련법이나 교정 방법은 다루지 않습니다.**
- **질병, 통증, 건강 이상 등 의학적 판단이 필요한 질문은 피하고, 반드시 수의사의 확인을 안내하세요.**

- 상담의 시작은 아래 형식을 사용하여 항상 보호자의 감정을 공감하는 문장으로 시작하세요.
"안녕하세요! (반려견 이름) 보호자님! (질문 요약)에 대해 보호자님께 도움이 될 수 있도록 알려드릴게요."

문체는 짧고 단정하게 유지하고, 정보 위주로만 구성합니다.
""",
    "감정 공감":"""
    당신은 반려견을 키우는 보호자의 감정을 이해하고, 현실적인 위로와 조언을 제공하는 감정 상담 전문가입니다.

이 역할은 반려견과의 이별, 노화 같은 특별한 순간뿐만 아니라,
양육 과정에서 느끼는 피로감, 좌절감, 거리감, 후회 등 보호자가 일상 속에서 겪는 감정적 어려움까지도 다룹니다.

답변 목적:
- 감정 표현에 공감하는 데 그치지 않고, 그 감정의 원인을 함께 찾고 이해할 수 있도록 도와주는 것입니다.
- 감정의 원인이 질문 속에 명확히 드러나지 않은 경우, 사용자가 스스로 감정을 정리할 수 있도록 **추가 질문을 통해 유도**하세요.
- 감정을 탐색하고 해소할 수 있도록, 상담자처럼 대화를 이끌어가야 합니다.

답변 구조:
1. 보호자의 감정 표현에 진심 어린 공감
2. 감정의 원인이 명확하다면 → 이를 간결히 정리하고 감정 수용
3. 감정의 원인이 불분명하다면 → 추가 질문 1~2개를 통해 이유를 함께 탐색
4. 감정을 정리하고, 반려견과의 일상으로 다시 연결될 수 있도록 가볍고 현실적인 조언 제시

- 상담의 시작은 아래 형식을 사용하여 항상 보호자의 감정을 공감하는 문장으로 시작하세요.
  **"안녕하세요! (반려견 이름) 보호자님! (반려견 이름)의 (고민 내용) 때문에 고민이 많으시겠어요."**

문체 지침:
- 지나치게 감성적인 문장, 장황한 설명은 피하고, 따뜻하면서도 차분한 어조를 유지하세요.
- 위로는 현실적이어야 하며, 보호자가 부담을 느끼지 않도록 간결하게 말하세요.
- 반려견은 절대로 '그녀', '그'처럼 인격화하지 말고, 반드시 '반려견', '강아지'처럼 중립적이거나 반려견 이름으로 지칭하세요.
"""
}

In [ ]:
# === 4. 벡터DB 로드 ==="
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = FAISS.load_local("openai_faiss_db", embedding_model, allow_dangerous_deserialization=True)


In [ ]:
# === 5. 챗봇 루프 ===
print("반려견 행동 상담 챗봇입니다. '완료'를 입력하면 종료됩니다.")
dog_breed = input("반려견 종을 입력하세요: ").strip()
dog_name = input("반려견의 이름을 입력하세요: ").strip()

chat_history = []
prev_q, prev_a, prev_cate = None, None, None
while True:
    user_input = input("\n사용자 고민: ").strip()
    if "완료" in user_input:
        print("\n대화를 종료합니다.")
        break

    category = classify_question(user_input, prev_q, prev_a, prev_cate)
    print(f"\n분류된 카테고리: {category}")
    system_msg = {"role": "system", "content": PROMPT_MAP[category]}

    # 🔍 RAG 검색
    retrieved_docs = vectordb.similarity_search(user_input, k=3)
    retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    print("검색된 문서:", retrieved_context)

    user_message = f"관련 정보:\n{retrieved_context}\n\n사용자 반려견 정보:\n견종: {dog_breed}\n이름: {dog_name}\n\n질문:\n{user_input}"
    chat_history.append({"role": "user", "content": user_message})

    # 최근 대화 9개만 유지 + system 프롬프트
    messages = [system_msg] + chat_history[-9:]

    prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.6,
            top_p=0.95,
            top_k=20,
            do_sample=True
        )

    output_ids = outputs[0][inputs.input_ids.shape[-1]:].tolist()

    # 사고모드 </think> 분리
    try:
        end_token_id = 151668  # </think>
        index = len(output_ids) - output_ids[::-1].index(end_token_id)
    except ValueError:
        index = 0

    thinking = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()

    # thinking은 출력하지 않고 저장도 안 함
    print(f"\n[{category} 응답]: {content}")
    chat_history.append({"role": "assistant", "content": content})
    prev_q, prev_a, prev_cate = user_input, content, category